In [210]:
### IMPORT NECESSARY PACKAGES ###
# First, we'll import the packages that we'll be using in the code
import random
import math
import numpy as np
import matplotlib.cm as colormaps
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import os
import sys
import time
import itertools
from scipy.integrate import odeint

Gconst=1.0 ######4.3*10**(-3)  ##pc⋅Msun^−1*(km/s)^2

In [211]:
### READ IN USER INPUTS ###
# Next, we'll define the number of particle, steps, and the timestep. This is all we will need to define!
n_particles = float(input('Number of particles:   ') or 5)
if n_particles < 1 or n_particles % 1.0 != 0:
    print('Error: Number of particles must be an integer greater than 0!')
    sys.exit()

n_particles=int(n_particles)


n_steps = float(input('Number of steps (divisible by 100):   ') or 1000)
if n_steps < 100 or n_steps % 100.0 != 0:
    print('Error: Number of steps must be an integer divisible by 100!')
    sys.exit()

n_steps=int(n_steps)


n_dim = int(input('Number of dimensions (2 or 3):   ') or 2)
if n_dim != 2 and n_dim != 3:
    print('Error: Number of dimensions must be 2 or 3!')
    sys.exit()

n_dim=int(n_dim)


timestep = float(input('Specify the timestep:   ') or 0.01)

if timestep < 0:
    print('Error: Timestep must be a positive number!')
    sys.exit()
    
    
##Choose to manually input masses or randomly generate masses    
random_mass=str(input('Randomly generate masses?:   ') or "yes")
if random_mass!="yes" and random_mass!="no":
    print('Error: Input choice must be yes or no!')

if random_mass=="no":
    masses=[]
    for i in range(n_particles):
        masses.append(float(input('m'+str(i+1)+':   ') or 1.))
        if masses[i]<0:
            print('Error: Mass must be a positive number!')
            sys.exit()
            

##Choose to manually input particle positions or randomly generate particle positions
random_pos=str(input('Randomly generate particle positions?:   ') or "yes")
if random_pos!="yes" and random_pos!="no":
    print('Error: Input choice must be yes or no!')

if random_pos=="no":
    x0=[]; y0=[]
    for i in range(n_particles):
        x0.append(float(input('x'+str(i+1)+':   ') or float(i)))
        if x0[i]>50.:
            print('Error: Position must be within 50!')
            sys.exit()
        y0.append(float(input('y'+str(i+1)+':   ') or float(i)))
        if y0[i]>50.:
            print('Error: Position must be within 50!')
            sys.exit()


Number of particles:   3
Number of steps (divisible by 100):   5000
Number of dimensions (2 or 3):   3
Specify the timestep:   0.01
Randomly generate masses?:   yes
Randomly generate particle positions?:   yes


In [212]:
### SETUP DIRECTORY FOR OUTPUT AND PARTICLES COLORS ###
# define the colors for each particle
cmap = colormaps.get_cmap('Set1')
colors = [cmap(i) for i in np.linspace(0,1,n_particles)]


# find working directory and make folder for images
curr_time = time.strftime("%Y-%m-%d-%H%M%S")
dirname = 'Nbody_'+str(curr_time)
os.makedirs(dirname)

In [213]:
### CHOOSE RANDOM INITIAL CONDITIONS FOR THE PARTICLES ###
# First, we'll assign each particle a random mass between 1 and 5
# This can be done using a such that when you change the number of particles above, you won't need to add extra lines of code!
min_mass, max_mass = 1, 10

if random_mass=="yes":
    i=0; masses = []
    while i < n_particles:
        masses.append(random.randint(min_mass,max_mass))
        i += 1
else:
    masses=masses

print("Particle masses: ")
print(masses)



# We'll do the same thing for the initial position. We'll choose a starting position on the grid between -50 and 50
max_pos = 5.

# Let's get the starting x positions
if random_pos=="yes":
    x0 = []
    i=0
    while i < n_particles:
        x0.append(2*max_pos*(random.random()-0.5))
        i += 1  
else:
    x0=x0

print("Starting x coordinates: ")
print(x0)



# Now for the y positions
if random_pos=="yes":
    y0 = []
    i=0
    while i < n_particles:
        y0.append(2*max_pos*(random.random()-0.5))
        i += 1
else:
    y0=y0

print("Starting y coordinates: ")
print(y0)



# set z-position to 0 if <3 dimensions
if n_dim == 2:
    z0 = []
    i=0
    while i < n_particles:
        z0.append(0)
        i += 1
elif n_dim == 3:
    z0 = []
    i=0
    while i < n_particles:
        z0.append(2*max_pos*(random.random()-0.5))
        i += 1 

print("Starting z coordinates: ")
print(z0)




# Let's just assume all the particles are starting from rest, so the initial velocity is 0. Can adjust the maximum component velocities by changing the value below

max_vel = 1.0
vx0 = []
i=0
while i < n_particles:
    vx0.append(2*max_vel*(random.random()-0.5))
    i += 1

print("Starting x velocity: ")
print(vx0)


vy0 = []
i=0
while i < n_particles:
    vy0.append(2*max_vel*(random.random()-0.5))
    i += 1

print("Starting y velocities: ")
print(vy0)


# set z-velocity to 0 if <3 dimensions
if n_dim == 2:
    vz0 = []
    i=0
    while i < n_particles:
        vz0.append(0)
        i += 1
elif n_dim == 3:
    vz0 = []
    i=0
    while i < n_particles:
        vz0.append(2*max_vel*(random.random()-0.5))
        i += 1

print("Starting z velocities: ")
print(vz0)

Particle masses: 
[1, 3, 9]
Starting x coordinates: 
[-2.9902786556582073, 4.465780990146843, 2.651902247893041]
Starting y coordinates: 
[-0.874017402417655, -0.16278105340585602, -4.2114208848364365]
Starting z coordinates: 
[-3.1677191060686916, 0.6497660236902059, -4.013041042182548]
Starting x velocity: 
[-0.4428646557907181, -0.7385591663003253, -0.3300422452554601]
Starting y velocities: 
[-0.0005113203907696207, -0.6576966234096289, -0.8658514927257115]
Starting z velocities: 
[-0.23991106869534295, -0.8387713863841244, 0.9611490107537635]


In [214]:
### DEFINE FUNCTIONS THAT WILL BE CALLED IN THE LOOP ###
# Our force function needs the target and object positions and masses. We will loop this function to get the combined force on the 'target' particle from all the other 'object' particles

def calc_force(m_t, m_o, x_t, x_o, y_t, y_o, z_t=0, z_o=0):
    # calculate the distance between the target and object (remember, we are doing one component at a time, so we will do this independently for x and y)
    dx = x_o-x_t
    dy = y_o-y_t
    dz = z_o-z_t

    pos = np.asarray([dx, dy, dz])
    r = math.sqrt(dx**2+dy**2+dz**2)


    # calculate the force between these particles
    f = Gconst*m_t*m_o / r**3 * pos


    # "soften" the gravity to ensure the force never gets too large, start doing this at r=sqrt(2)
    if np.sqrt(f[0]**2+f[1]**2+f[2]**2) > m_t*m_o/2:
        f = Gconst*m_t*m_o/4. * f/np.sqrt(f[0]**2+f[1]**2+f[2]**2)

    fx = f[0]
    fy = f[1]
    fz = f[2]

    return fx, fy, fz

  

# Now, we'll have a function that takes the total force in a particular direction and determine the change in distance and velocity in that direction

def calc_position(d_t, v_t, t):
    # we calculate the position change using the (symplectic) leap frog method
    # technically, this is assuming that the velocity is calculate at n*(t-0.5)
    
    # first, we let the particle drift at the current velocity 
    d = d_t + v_t*t

    return d


def calc_velocity(f, m_t, v_t, t):
    # then we calculate the acceleration on the particle from the force
    if m_t == 0:
        a = 0

    else:
        a = f/m_t
    
    # now we calculate the new velocity, which will kick the particle on the next step
    v = v_t + a*t
    
    return v

    

##Try using scipy ODEint to calculate the trajectory##
#def integrate_trajectory(position, t, force, mass):
#    position, velocity=A
#    dAdt=[velocity, force/mass]
#    
#    return dAdt



# We will also keep track at the energy at every step. The total energy *should* be conserved in an ideal N-body code
def calc_energy(masses, pos_xs, pos_ys, pos_zs, vel_xs, vel_ys, vel_zs):
    KEs=[]
    PEs=[]

    i=0
    while i < n_particles:
        KEs.append(0.5*masses[i]*(vel_xs[i]**2 + vel_ys[i]**2 + vel_zs[i]**2))

        PE_i=[]
        j=0
        while j < n_particles:
            if i != j:
                PE_i.append(-1*Gconst*masses[i]*masses[j]/math.sqrt((pos_xs[i]-pos_xs[j])**2 + (pos_ys[i]-pos_ys[j])**2 + (pos_zs[i]-pos_zs[j])**2))
            j+=1

        PEs.append(sum(PE_i))
        i+=1

    Ekin=sum(KEs)
    Epot=sum(PEs)
    Etot=Ekin+Epot

    return Ekin, Epot, Etot 




# Now, a function for writing the data at each step to a file
def write_data(t, Ekin, Epot, Etot, masses, xs, ys, zs, vxs, vys, vzs):
    f = open(dirname+'/output.dat', 'a')
    f.write('\n%s %s %s %s ' % (fmt(t), fmt(Ekin), fmt(Epot), fmt(Etot)))
    for i in range(n_particles):
        f.write('%s ' % fmt(masses[i]))
        f.write('%s ' % fmt(xs[i]))
        f.write('%s ' % fmt(ys[i]))
        f.write('%s ' % fmt(zs[i]))
        f.write('%s ' % fmt(vxs[i]))
        f.write('%s ' % fmt(vys[i]))
        f.write('%s ' % fmt(vzs[i]))

    f.close() 

In [215]:
### FORMAT OUTPUT FILE AND WRITE HEADER ###
# Write the header and starting conditions to the output file
def fmt(value):
    return "%.2f" % value

# this should give us the correct headers, despite how many particles we have...

numbers=np.arange(n_particles)+1
numbers=[str(i) for i in numbers]

mass_labels=['m']
pos_labels=['x','y','z']
vel_labels=['xv','yv','zv']

mass_names=map(''.join, itertools.product(numbers, mass_labels))
pos_names=map(''.join, itertools.product(numbers, pos_labels))
vel_names=map(''.join, itertools.product(numbers, vel_labels))

mass_names=[i[::-1] for i in mass_names]
pos_names=[i[::-1] for i in pos_names]
vel_names=[i[::-1] for i in vel_names]

pos_names = np.reshape(pos_names, (n_particles,3))
vel_names = np.reshape(vel_names, (n_particles,3))

f = open(dirname+'/output.dat', 'w')
f.write('#t Ekin Epot Etot ')
for i in range(n_particles):
    f.write('%s ' % mass_names[i])

    for dim in pos_names[i]:
        f.write('%s ' % dim)

    for dim in vel_names[i]:
        f.write('%s ' % dim)

f.close()

In [216]:
# write starting data
Ekin0, Epot0, Etot0 = calc_energy(masses, x0, y0, z0, vx0, vy0, vz0)

write_data(0, Ekin0, Epot0, Etot0, masses, x0, y0, z0, vx0, vy0, vz0)
x_step=x0[:]
y_step=y0[:]
z_step=z0[:]

vx_step=vx0[:]
vy_step=vy0[:]
vz_step=vz0[:]


###Calculate starting force and move velocity back by 0.5dt
#i=0   # counter for the target particle
#
#while i < n_particles:
#
#    forces_x=[]   # this list will temporaily hold the forces on the target particle
#    forces_y=[]   # this list will temporaily hold the forces on the target particle
#    forces_z=[]
#     
#
#    j=0   # counter for the object particles
#    while j < n_particles:
#
#        # NOTE: we don't want to calculate the force between the particle and itself
#
#        if i != j:
#            # calculate distance between the particles
#            r = math.sqrt((x_step[i]-x_step[j])**2+(y_step[i]-y_step[j])**2 + (z_step[i]-z_step[j])**2)
#
#            # remember, i is the target particle and j is the object particle
#
#            fx, fy, fz = calc_force(masses[i], masses[j], x_step[i], x_step[j], y_step[i], y_step[j], z_step[i], z_step[j])
#
#            # we hold onto the force contributuions from each object particle on the target particle
#
#            forces_x.append(fx)
#            forces_y.append(fy)
#            forces_z.append(fz)
#
#        j += 1
#
#    
#
#    # let's now sum up the forces on the target particle! 
#
#    fx = sum(forces_x)
#    fy = sum(forces_y)
#    fz = sum(forces_z)
#
#    # now, let's see how the position and velocity change of the target particle by calling the "calc_trajcetory" function
#
#    vx = calc_velocity(fx, masses[i], vx_step[i], 0.5*timestep)  ##These are v1/2, the half step velocities
#    vy = calc_velocity(fy, masses[i], vy_step[i], 0.5*timestep)
#    vz = calc_velocity(fz, masses[i], vz_step[i], 0.5*timestep)
#    
#    x=calc_position(x_step[i], vx, timestep)
#    y=calc_position(y_step[i], vy, timestep)
#    z=calc_position(z_step[i], vz, timestep)
#    
#    
#    # lastly, we hold on to these values in our "step" list
#
#    x_temp.append(x)
#    vx_temp.append(vx)
#
#    y_temp.append(y)
#    vy_temp.append(vy)
#
#    z_temp.append(z)
#    vz_temp.append(vz)
#
#        
#    i += 1
#
#
#x_step = x_temp[:]    ##These are x1
#y_step = y_temp[:]
#z_step = z_temp[:]
#
#vx_step = vx_temp[:]   ##These are v1/2, the half step velocities
#vy_step = vy_temp[:]
#vz_step = vz_temp[:]




In [217]:
#################

### MAIN LOOP ###

#################

s=0   # counter for the number of steps

while s < n_steps:
    s+=1

    ## see if any of the particles collided and combine them (assuming the mass is proportional to the radius)
#
    #i=0
#
    #while i < n_particles:
#
    #    j=0
    #    while j < n_particles:
    #        if i != j:
    #            r = math.sqrt((x_step[i]-x_step[j])**2+(y_step[i]-y_step[j])**2 + (z_step[i]-z_step[#j])**2)
#
    #            if r < 0.05*(masses[i]+masses[j]):
#
    #                # COLLISION! Make the larger particle eat the smaller particle, and find the #new velocities using conservation of momentum. 
#
    #                m_tot = masses[i]+masses[j]
#
    #                vx_tot = (masses[i]*vx_step[i]+masses[j]+vx_step[j])/m_tot
    #                vy_tot = (masses[i]*vy_step[i]+masses[j]+vy_step[j])/m_tot
    #                vz_tot = (masses[i]*vz_step[i]+masses[j]+vz_step[j])/m_tot
#
    #                if masses[i] >= masses[j]:
    #                    masses[i], masses[j] = m_tot, 0
#
    #                    vx_step[i], vx_step[j] = vx_tot, 0
    #                    vy_step[i], vy_step[j] = vy_tot, 0
    #                    vz_step[i], vz_step[j] = vz_tot, 0
#
    #                elif masses[i] < masses[j]:
    #                    masses[j], masses[i] = m_tot, 0
#
    #                    vx_step[j], vx_step[i] = vx_tot, 0
    #                    vy_step[j], vy_step[i] = vy_tot, 0
    #                    vz_step[j], vz_step[i] = vz_tot, 0
#
    #        j += 1
#
    #    i += 1

                    

    # we'll now make a new list to hold the position and velocity values for the next step

    x_temp=[]
    y_temp=[]
    z_temp=[]

    vx_temp=[]
    vy_temp=[]
    vz_temp=[]



    # first, we do the force calculation for each particle in the x-direction and calculate the x-trajectory of each particle

    i=0   # counter for the target particle

    while i < n_particles:

        forces_x=[]   # this list will temporaily hold the forces on the target particle
        forces_y=[]   # this list will temporaily hold the forces on the target particle
        forces_z=[]
     

        j=0   # counter for the object particles
        while j < n_particles:

            # NOTE: we don't want to calculate the force between the particle and itself

            if i != j:
                # calculate distance between the particles
                r = math.sqrt((x_step[i]-x_step[j])**2+(y_step[i]-y_step[j])**2 + (z_step[i]-z_step[j])**2)

                # remember, i is the target particle and j is the object particle

                fx, fy, fz = calc_force(masses[i], masses[j], x_step[i], x_step[j], y_step[i], y_step[j], z_step[i], z_step[j])

                # we hold onto the force contributuions from each object particle on the target particle

                forces_x.append(fx)
                forces_y.append(fy)
                forces_z.append(fz)

            j += 1

    

        # let's now sum up the forces on the target particle! 

        fx = sum(forces_x)
        fy = sum(forces_y)
        fz = sum(forces_z)

        

        # now, let's see how the position and velocity change of the target particle by calling the "calc_trajcetory" function

        vx = calc_velocity(fx, masses[i], vx_step[i], 0.5*timestep)
        vy = calc_velocity(fy, masses[i], vy_step[i], 0.5*timestep)
        vz = calc_velocity(fz, masses[i], vz_step[i], 0.5*timestep)
        
        x=calc_position(x_step[i], vx, timestep)
        y=calc_position(y_step[i], vy, timestep)
        z=calc_position(z_step[i], vz, timestep)
        
          
        #print(vx, vy, vz)
        #print(x, y, z)
        
        
        # lastly, we hold on to these values in our "step" list

        x_temp.append(x)
        vx_temp.append(vx)

        y_temp.append(y)
        vy_temp.append(vy)

        z_temp.append(z)
        vz_temp.append(vz)

        
        i += 1



    # now that we have calculated the new position of all the particles, we will overwrite the last step and save the data for this step

    x_step = x_temp[:]
    y_step = y_temp[:]
    z_step = z_temp[:]
    
    vx_step = vx_temp[:]
    vy_step = vy_temp[:]
    vz_step = vz_temp[:]
    
    
    # first, we do the force calculation for each particle in the x-direction and calculate the x-trajectory of each particle

    i=0   # counter for the target particle

    while i < n_particles:

        forces_x=[]   # this list will temporaily hold the forces on the target particle
        forces_y=[]   # this list will temporaily hold the forces on the target particle
        forces_z=[]
     

        j=0   # counter for the object particles
        while j < n_particles:

            # NOTE: we don't want to calculate the force between the particle and itself

            if i != j:
                # calculate distance between the particles
                r = math.sqrt((x_step[i]-x_step[j])**2+(y_step[i]-y_step[j])**2 + (z_step[i]-z_step[j])**2)

                # remember, i is the target particle and j is the object particle

                fx, fy, fz = calc_force(masses[i], masses[j], x_step[i], x_step[j], y_step[i], y_step[j], z_step[i], z_step[j])

                # we hold onto the force contributuions from each object particle on the target particle

                forces_x.append(fx)
                forces_y.append(fy)
                forces_z.append(fz)

            j += 1

    

        # let's now sum up the forces on the target particle! 

        fx = sum(forces_x)
        fy = sum(forces_y)
        fz = sum(forces_z)
        
        # now, let's see how the position and velocity change of the target particle by calling the "calc_trajcetory" function

        vx = calc_velocity(fx, masses[i], vx_step[i], 0.5*timestep)
        vy = calc_velocity(fy, masses[i], vy_step[i], 0.5*timestep)
        vz = calc_velocity(fz, masses[i], vz_step[i], 0.5*timestep)
        
          
        # lastly, we hold on to these values in our "step" list
        vx_temp.append(vx)
        vy_temp.append(vy)
        vz_temp.append(vz)

        
        i += 1


    vx_step = vx_temp[:]
    vy_step = vy_temp[:]
    vz_step = vz_temp[:]

     

    # we now have lists with the new positions and velocities, let's append them to the full list of values

    Ekin_step, Epot_step, Etot_step = calc_energy(masses, x_step, y_step, z_step, vx_step, vy_step, vz_step)



    # write data to file
    write_data(s*timestep, Ekin_step, Epot_step, Etot_step, masses, x_step, y_step, z_step, vx_step, vy_step, vz_step)

    


    # plot where the particles are at each step

    #fig=plt.figure(figsize = (12,12))
#
    #if n_dim == 2:
#
    #    ax = fig.add_subplot(111)
#
    #    ax.set_xlim(-50,50)  ##-max_pos,max_pos
#
    #    ax.set_ylim(-50,50) ##-max_pos,max_pos
#
    #    ax.set_xlabel('x', fontsize=15)
#
    #    ax.set_ylabel('y', fontsize=15)
#
    #    for i in range(n_particles):
#
    #        ax.scatter(x_step[i], y_step[i], color=colors[i], s=8*masses[i])
#
    #elif n_dim == 3:
#
    #    ax = fig.add_subplot(111, projection='3d')
#
    #    ax.set_xlim(-50,50)
#
    #    ax.set_ylim(-50,50)
#
    #    ax.set_zlim(-50,50)
#
    #    ax.set_xlabel('x', fontsize=15)
#
    #    ax.set_ylabel('y', fontsize=15)
#
    #    ax.set_zlabel('z', fontsize=15)
#
    #    for i in range(n_particles):
#
    #        ax.scatter(x_step[i], y_step[i], z_step[i], color=colors[i], s=8*masses[i])
#
    #plt.savefig(dirname+'/nbody_'+str(s).zfill(4)+'.png')
#
    #plt.close()



    # see how far into the simulation we are

    if s % (n_steps/100.0) == 0:

        sys.stderr.write('\r %{0} step:{1}'.format(100*float(s) / n_steps, s))


print('\nDone!')

SyntaxError: invalid syntax (<ipython-input-217-3bf0837b737c>, line 23)